# Importing my own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import


In [1]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/code/mybook-dashboard'

In [2]:
# Improt packages
import pandas as pd
import json 
import numpy as np

# Import functions from apps folder
# from apps.collect_data import *

pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 80)

In [3]:
mybooksgr = pd.read_csv("assets/goodreads_library_export.csv")
# mybooksgr = mybooksgr.rename(columns=lambda x: x.replace(' ', '_'))


In [7]:
import nest_asyncio
nest_asyncio.apply()

from apps.async_googleapi import book_info_add, asyncio
from apps.api import api_key
pimydf = asyncio.run(book_info_add(mybooksgr, api_key))

The People on Platform 5
State of Paradise
Death on Cromer Beach (DS Ashley Knight #1)
De syv søstre
Billy Summers
Elskeren
The Only One Left
Daughter of the Moon Goddess (The Celestial Kingdom Duology #1)
Heart of the Sun Warrior (The Celestial Kingdom Duology, #2)
House of Earth and Blood (Crescent City, #1)
How to Age Disgracefully
I Hope This Finds You Well
The Case Against Sugar
The Housemaid Is Watching (The Housemaid, #3)
This Summer Will Be Different
The Paradise Problem
Jenny
The Poisonwood Bible
Demon Copperhead
The Great Alone
Blue Sisters
All the Colors of the Dark
Skråpånatta
The Eighth Life: for Brilka
The Charity Shop Detective Agency (The Charity Shop Detective Agency, #1)
The Dream Home
The Last Devil to Die (Thursday Murder Club, #4)
Then She Vanishes
The Wrong Sister
Life for Sale
The Night Nanny
Crossroads
El viento que arrasa
The Day I Die: The Untold Story of Assisted Dying in America
Darling Girls
Funny Story
The Reappearance of Rachel Price
The List of Suspiciou

# Merge dataframes 
mybooks = pd.merge(mybooksgr,
                     pimydf,
                     on='Title', 
                     suffixes = ('_Goodreads', '_GoogleBooks'), 
                     how='left')

In [8]:

from apps.dataimport import dataprep
nmyreads = dataprep(mybooksgr, pimydf)

In [9]:
import pickle
nmyreads.to_pickle("assets/my_books.pkl")

# Data cleaning books steps, PS this is already run above
- page count categories
- filter if book is read of not 

In [35]:
# Page count category variable

def categorize_pages(number_of_pages):
    if number_of_pages >= 100 and number_of_pages <= 249:
        return '100-249'
    elif number_of_pages >= 250 and number_of_pages <= 349:
        return '250-349'
    elif number_of_pages >= 350 and number_of_pages <= 449:
        return '350-449'
    elif number_of_pages >= 450 and number_of_pages <= 599:
        return '450-599'
    elif number_of_pages >= 600 and number_of_pages <= 749:
        return '600-749'
    elif number_of_pages >= 750 and number_of_pages <= 999:
        return '750-999'
    else:
        return '1000+'

# Apply the categorize_pages function to create the 'Page_Cat' column
mybooks['Page_Cat'] = mybooks['Number_of_Pages'].apply(categorize_pages)

# Define the desired order of categories
category_order = ['100-249', '250-349', '350-449', '450-599', '600-749', '750-999', '1000+']

# Convert the 'Page_Cat' column to a categorical variable with the specified order
mybooks['Page_Cat'] = pd.Categorical(mybooks['Page_Cat'], categories=category_order, ordered=True)


In [36]:
# drop duplicates
mybooks = mybooks.drop_duplicates(subset=['Title', 'Author'])

In [38]:
# Create year and quarter read variable 
#  Impute data_added where date_read  is na
mybooks['Date_Read'] = np.where(mybooks['Date_Read'].isnull() & mybooks['Read_Count']==1, mybooks['Date_Added'], mybooks['Date_Read'])

# Convert 'Date_Read' column to datetime type
mybooks['Date_Read'] = pd.to_datetime(mybooks['Date_Read'], format='mixed')

# Extract year and quarter from 'Date_Read' column
mybooks['Year'] = mybooks['Date_Read'].dt.year
mybooks['Quarter'] = mybooks['Date_Read'].dt.quarter

# Create a new column combining year and quarter
mybooks['Year_Quarter'] = np.where(mybooks['Date_Read'].notnull(), mybooks['Year'].astype(str) + '-Q' + mybooks['Quarter'].astype(str), np.nan)
# Replace '.0' in the Year_Quarter column with an empty string
mybooks['Year_Quarter'] = mybooks['Year_Quarter'].fillna('').str.replace('.0', '')

# Convert Year_Quarter to categorical variable
mybooks['Year_Quarter'] = pd.Categorical(mybooks['Year_Quarter'], ordered=True)

In [41]:
# filter na in publication year and make column publication year integer 
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].fillna( 0)
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].astype(int)

In [43]:
# Making sure all na is set as np.nan and not as a string variable (had this issue with one variable)
import numpy as np
mybooks = mybooks.replace('nan', np.nan)
mybooks = mybooks.replace('NaN', np.nan)

# Collect book topics 

In [14]:
# Collect topics for my own books from OLapi
from apps.collect_data import get_book_topics
my_topics = get_book_topics(mybooksgr)

In [15]:
dict = mybooksgr[['Author','Title']].to_dict()
list(set(dict['Title'].values()))
    

['Mormor danset i regnet',
 'Things We Left Behind (Knockemout, #3)',
 'We Were Liars',
 'Strange Weather in Tokyo',
 'Such a Fun Age',
 'The Prison Healer (The Prison Healer, #1)',
 'Min europeiska familj: De senaste 54 000 åren',
 'The Teacher',
 "Factfulness: Ten Reasons We're Wrong About the World – and Why Things Are Better Than You Think",
 'Life for Sale',
 'The Girlfriend',
 'The Dutch House',
 "I'm a Fan",
 'The Art of Statistics: How to Learn from Data',
 'Free Lunch Thinking: How Economics Ruins the Economy',
 'In the Time of the Butterflies',
 'Courage To Be Disliked / The Road Back to You / Mindset',
 'Never Lie',
 'A Gentleman in Moscow',
 'Earthlings',
 'Neverwhere (London Below, #1)',
 '1Q84 Book 1 (1Q84, #1)',
 'Tomorrow, and Tomorrow, and Tomorrow',
 'Anxious People',
 "Professional Pattern Grading for Women'S, Men'S, and Children's Apparel",
 'We Are All Birds of Uganda',
 'Convenience Store Woman',
 'A Storm of Swords (A Song of Ice and Fire, #3)',
 'Icebreaker (UCM

In [16]:
df = pd.DataFrame(dict)
df

,Author,Title
0,Clare Pooley,The People on Platform 5
1,Laura van den Berg,State of Paradise
2,Ross Greenwood,Death on Cromer Beach (DS Ashley Knight #1)
3,Lucinda Riley,De syv søstre
4,Stephen King,Billy Summers
...,...,...
498,Ken Follett,"The Evening and the Morning (Kingsbridge, #0)"
499,Anthony Doerr,All the Light We Cannot See
500,George Orwell,1984
501,Cristina García,Monkey Hunting: A Novel (Ballantine Reader's C...


In [17]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)

In [41]:
mybooksgr.query('`Read Count` ==1').head(2).to_csv("assets/my_books_test.csv", index=False)